### 6.3 Семинар: cлой нормализации
#### 2 из 11 шагов

В этом уроке мы детально изучим слои нормализации.

Самая популярная версия слоя нормализации - слой нормализации "по батчу" (batch-norm слой).

В данном шаге вам требуется реализовать функцию батч-нормализации без использования стандартной функции со следующими упрощениями:

- Параметр Бета принимается равным 0.
- Параметр Гамма принимается равным 1.
- Функция должна корректно работать только на этапе обучения.
- Вход имеет размерность число элементов в батче * длина каждого инстанса.
 
Очень внимательно посмотрите на определение функции, вычисляющей std.

Sample Input: anything

Sample Output: True

In [1]:
import numpy as np
import torch
import torch.nn as nn

def custom_batch_norm1d(input_tensor, eps):
    mean = input_tensor.mean(dim=0)
    var = input_tensor.var(dim=0, unbiased=False)
    
    normed_tensor = (input_tensor - mean) / torch.sqrt(var + eps)
    return normed_tensor


input_tensor = torch.Tensor([[0.0, 0, 1, 0, 2], [0, 1, 1, 0, 10]])
batch_norm = nn.BatchNorm1d(input_tensor.shape[1], affine=False)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# import numpy as np
# all_correct = True
# for eps_power in range(10):
#     eps = np.power(10., -eps_power)
#     batch_norm.eps = eps
#     batch_norm_out = batch_norm(input_tensor)
#     custom_batch_norm_out = custom_batch_norm1d(input_tensor, eps)

#     all_correct &= torch.allclose(batch_norm_out, custom_batch_norm_out)
#     all_correct &= batch_norm_out.shape == custom_batch_norm_out.shape
# print(all_correct)

#### 3 из 11 шагов

Немного обобщим функцию с предыдущего шага - добавим возможность задавать параметры Бета и Гамма.

На данном шаге вам требуется реализовать функцию батч-нормализации без использования стандартной функции со следующими упрощениями:

Функция должна корректно работать только на этапе обучения.
Вход имеет размерность число элементов в батче * длина каждого инстанса.

Sample Input: anything

Sample Output: True

In [2]:
import torch
import torch.nn as nn

input_size = 7
batch_size = 5
input_tensor = torch.randn(batch_size, input_size, dtype=torch.float)

eps = 1e-3

def custom_batch_norm1d(input_tensor, weight, bias, eps):
    # Шаг 1: Вычисление среднего значения
    mean = input_tensor.mean(dim=0)
    
    # Шаг 2: Вычетание среднего значения
    centered_input = input_tensor - mean
    
    # Шаг 3: Вычисление дисперсии и нормализация
    variance = centered_input.pow(2).mean(dim=0)
    stddev = torch.sqrt(variance + eps)
    normalized_input = centered_input / stddev
    
    # Шаги 4 и 5: Применение весового коэффициента и смещения
    normed_tensor = weight * normalized_input + bias
    
    return normed_tensor

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# batch_norm = nn.BatchNorm1d(input_size, eps=eps)
# batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
# batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
# batch_norm_out = batch_norm(input_tensor)
# custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)
# print(torch.allclose(batch_norm_out, custom_batch_norm_out, 1e-3) \
#       and batch_norm_out.shape == custom_batch_norm_out.shape)

#### 4 из 11 шагов

Избавимся еще от одного упрощения - реализуем работу слоя батч-нормализации на этапе предсказания.

На этом этапе вместо статистик по батчу будем использовать экспоненциально сглаженные статистики из истории обучения слоя.

В данном шаге вам требуется реализовать полноценный класс батч-нормализации без использования стандартной функции, принимающий на вход двумерный тензор. Осторожно, расчёт дисперсии ведётся по смещенной выборке, а расчет скользящего среднего по несмещенной.

Sample Input: anything

Sample Output: True

In [7]:
import torch
import torch.nn as nn

# Настройки
input_size = 3
batch_size = 5
eps = 1e-1

class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum
        self.running_mean = torch.zeros(weight.shape[0])
        self.running_var = torch.ones(weight.shape[0])
        self.is_training = True

    def __call__(self, input_tensor):
        if self.is_training:
            # Расчёт текущих средних и дисперсий (несмещенная)
            batch_mean = input_tensor.mean(dim=0)
            batch_var = ((input_tensor - batch_mean) ** 2).mean(dim=0)
            # batch_var = torch.var(input_tensor, dim=0, unbiased=False)
            
            # Обновление экспоненциальных средних (смещенная оценка дисперсии)
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * batch_mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * batch_var * (batch_size / (batch_size - 1))

            # Нормализация
            normed_tensor = (input_tensor - batch_mean) / torch.sqrt(batch_var + self.eps) * self.weight + self.bias
        else:
            # Расчет и накопление экспоненциально сглаженных средних и дисперсий
            normed_tensor = (input_tensor - self.running_mean) / torch.sqrt(self.running_var + self.eps)
            # Применение весов и смещений
            normed_tensor = self.weight * normed_tensor + self.bias

        return normed_tensor

    def eval(self):
        self.is_training = False


# Стандартный слой батч-нормализации
batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

# Пользовательский слой батч-нормализации
custom_batch_norm1d = CustomBatchNorm1d(
    batch_norm.weight.data,
    batch_norm.bias.data,
    eps,
    batch_norm.momentum
)

In [109]:

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape

print(all_correct)
print("custom_output\n", custom_output)
print("norm_output\n", norm_output)

True
custom_output
 tensor([[ 0.8354,  0.6386, -1.1657],
        [ 0.3218,  0.4137,  2.2445],
        [-1.0693,  0.7540, -0.8937],
        [-0.4910,  0.5366,  1.5385],
        [ 0.7688,  0.2119, -2.7308]])
norm_output
 tensor([[ 0.8354,  0.6386, -1.1657],
        [ 0.3218,  0.4137,  2.2445],
        [-1.0693,  0.7540, -0.8937],
        [-0.4910,  0.5366,  1.5385],
        [ 0.7688,  0.2119, -2.7308]], grad_fn=<NativeBatchNormBackward0>)


#### 6 из 11 шагов

Как вы могли убедиться, реализовать батч-норм слой на этапе предсказания не так просто, поэтому в дальнейших шагах этого урока мы больше не будем требовать реализовать эту часть.
 
Слой батч-нормализации существует для входа любой размерности.

В данном шаге рассмотрим его для входа из многоканальных двумерных тензоров, например, изображений.

Если вытянуть каждый канал картинки в вектор, то вход будет трехмерным:

- количество картинок в батче
- число каналов в каждой картинке
- число пикселей в картинке

picture​

Процесс нормализации:

- Вход разбивается на срезы, параллельные синей части. То есть, каждый срез - это все пиксели всех изображений по одному из каналов.
- Для каждого среза считаются мат. ожидание и дисперсия.
- Каждый срез нормализуется независимо.
 

На данном шаге вам предлагается реализовать батч-норм слой для четырехмерного входа (например, батч из многоканальных двумерных картинок) без использования стандартной реализации со следующими упрощениями:

- Параметр Бета = 0.
- Параметр Гамма = 1.
- Функция должна корректно работать только на этапе обучения.

Sample Input: anything

Sample Output: True

In [6]:
import torch
import torch.nn as nn

eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)


def custom_batch_norm2d(input_tensor, eps):
    #mean = input_tensor.mean(dim=(0, 2, 3), keepdim=True)
    mean = torch.tensor([torch.mean(input_tensor[:,i,:,:]) for i in range(input_tensor.shape[1])])
    mean = mean.unsqueeze(0).unsqueeze(2).unsqueeze(3)
    # var = input_tensor.var(dim=(0, 2, 3), unbiased=False, keepdim=True)
    var = torch.tensor([torch.var(input_tensor[:,i,:,:], unbiased=False) for i in range(input_tensor.shape[1])])
    var = var.unsqueeze(0).unsqueeze(2).unsqueeze(3)
    
    normalized_tensor = (input_tensor - mean) / torch.sqrt(var + eps)
    
    return normalized_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = batch_norm_2d(input_tensor)
custom_output = custom_batch_norm2d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True


Объяснение шагов:
- Вычисление среднего значения (mean): Мы используем метод .mean() для вычисления среднего значения по всем изображениям в батче, а также по высоте и ширине каждого изображения. Параметры dim=(0, 2, 3) указывают, что средние будут рассчитаны по первым трем измерениям (то есть по батчу, высоте и ширине соответственно).keepdim=True сохраняет размеры тензора, чтобы он мог быть правильно использован при вычитании.
- Вычисление дисперсии (var): Аналогично среднему значению, мы вычисляем дисперсию для каждого канала, используя метод .var(). Здесь важно использовать параметр unbiased=False, так как стандартная реализация батч-нормализации использует несмещенную оценку дисперсии.
- Нормализация: После того как мы получили среднее и дисперсию, выполняем нормализацию каждого элемента тензора, вычитая среднее и деля на корень из дисперсии плюс небольшое число eps для числовой стабильности.

Из-за того, что платформа не поддерживает листы в dim=, пришлось искать обходные пути. Итого, нашёл три способа.

**Способ 1**, не поддерживвается платформой.
`mean = input_tensor.mean(dim=(0, 2, 3), keepdim=True)`
Аналогично для var (не забываем unbiased=False), и для normed_tensor - совсем просто.
Попробуйте этот способ, полагаю, знать его надо.

**Способ 2**, с конкатенацией через  .cat и .unsqueeze. Имхо коряво, но для понимания самое то.
Оъясню опять-таки только для mean:

Цикл по каналам:
`for i in range(input_tensor.shape[1]):` Цикл проходит по каждому каналу входного тензора (input_tensor).

Внутри - вычисление среднего по каналу:
`torch.mean(input_tensor[:,i,:,:])`. Вычисляет среднее значение всех элементов в i-м канале (по всем элементам batch, "высоте" и "ширине"). Результат — скалярное значение (одно число), представляющее среднее значение по каналу.

Добавление размерности:
unsqueeze(0): Добавляет новую ось (размерность) на нулевую позицию, делая скалярное значение тензором с размерностью (1,).

Объединение тензоров:
`torch.cat([...], dim=0)`: Объединяет полученные тензоры среднего значения для всех каналов вдоль первой оси (ось "batch"). Результат — тензор mean с размерностью (input_channels,).

Для var всё аналогично, не забываем unbiased=False.

Для normed_tensor - unsqueeze(1) и dim=1 (это подсказка не моя, а команды курса).
 

**Способ 3**, без склейки, простым преобразованием листа в тензор.

`mean = torch.tensor([torch.mean(input_tensor[:,i,:,:]) for i in range(input_tensor.shape[1])])`

Для var всё аналогично, не забываем unbiased=False.

Для normed_tensor применяем .stack - `torch.stack([...], dim=1)`. Создаём новый тензор, "складывая" тензоры из списка вдоль второй оси (dim=1).

#### 7 из 11 шагов

Идея, лежащая в основе слоя нормализации "по каналу", что сеть должна быть независимой от контраста исходного изображения.

Нормализация "по каналу" работает независимо по каждому изображению батча.

На этом шаге вам предлагается реализовать нормализацию "по каналу" без использования стандартного слоя со следующими упрощениями:
- Параметр Бета = 0.
- Параметр Гамма = 1.
- Требуется реализация только этапа обучения.
- Нормализация делается по всем размерностям входа, кроме нулевой.

**Обратите внимание, что размерность входа на данном шаге не фиксирована.**

Уточним, что в слое нормализации "по каналу" статистики считаются по всем размерностям, кроме нулевой.

Sample Input: anything

Sample Output: True

In [16]:
import torch
import torch.nn as nn


eps = 1e-10


def custom_layer_norm(input_tensor, eps):
    dims = list(range(1, len(input_tensor.size())))
    mean = input_tensor.mean(dims, keepdim=True)
    var = input_tensor.var(dims, unbiased=False, keepdim=True)
    
    normed_tensor = (input_tensor - mean) / torch.sqrt(var + eps)
    
    return normed_tensor

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True


1. Определение осей для расчета статистики:
```dims = list(range(1, len(input_tensor.size())))```

Мы создаем список индексов всех размерностей, начиная с 1-й оси (пропуская первую ось, соответствующую батчу).

2. асчет среднего значения:```mean = input_tensor.mean(dims, keepdim=True)```

Среднее значение рассчитывается по всем указанным осям, но благодаря параметру keepdim=True сохраняются те же самые размеры, что и у исходного тензора.

3. Расчет дисперсии:```var = input_tensor.var(dims, unbiased=False, keepdim=True)```

Дисперсия также считается по тем же осям, причем используется несмещенная оценка дисперсии (unbiased=False), что соответствует поведению стандартной реализации.

4. Нормализация:```normed_tensor = (input_tensor - mean) / torch.sqrt(var + eps)```

Нормализация выполняется путем вычитания среднего значения и деления на квадратный корень из дисперсии плюс малую константу eps для численной устойчивости.


In [19]:
import torch
import torch.nn as nn


eps = 1e-10


def custom_layer_norm(input_tensor, eps):
    # Определяем количество каналов (первая размерность после батча)
    num_dims = len(input_tensor.size())
    
    # Создадим списки для хранения средних значений и дисперсий
    means = []
    vars = []
    
    # Проходим по каждому элементу в батче
    for i in range(input_tensor.size(0)):
        # Берём срез по текущей позиции в батче
        slice_mean = torch.mean(input_tensor[i])
        means.append(slice_mean)
        
        # Рассчитываем дисперсию для текущего элемента батча
        slice_var = torch.var(input_tensor[i], unbiased=False)
        vars.append(slice_var)
    
    # Преобразуем списки в тензоры
    means_tensor = torch.stack(means)
    vars_tensor = torch.stack(vars)
    
    # Приводим размеры тензоров к нужному виду
    means_tensor = means_tensor.view(-1, *([1] * (num_dims - 1)))
    vars_tensor = vars_tensor.view(-1, *([1] * (num_dims - 1)))
    
    # Выполним нормализацию
    normed_tensor = (input_tensor - means_tensor) / torch.sqrt(vars_tensor + eps)
    
    return normed_tensor



# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    # print(norm_output)
    # print(custom_output)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True


**Не по каналу, а по батчу.** По каналу было прошлый раз. Сейчас проход по нулевой размерности input_tensor.

@Илья_Шишов, да, проходим по батчу, а усредняем каналы. Возможно стоило написать "по каналам", раз уж прямой перевод "канальная нормализация" звучит не очень. Но с переводами всегда сложно.

Все операции выполняются для размерности 0. `for j in range(0, input.size()[0])`

Разбор изменений:

Цикл по батчу:
```
for i in range(input_tensor.size(0)):
    slice_mean = torch.mean(input_tensor[i])
    means.append(slice_mean)
    
    slice_var = torch.var(input_tensor[i], unbiased=False)
    vars.append(slice_var)
```
Теперь цикл проходит по каждому элементу в батче (нулевое измерение), а не по каналам. Соответственно, для каждого элемента батча мы вычисляем среднее значение и дисперсию.

Приведение форматов:
```
means_tensor = means_tensor.view(-1, *([1] * (num_dims - 1)))
vars_tensor = vars_tensor.view(-1, *([1] * (num_dims - 1)))
```
Формируем тензоры таким образом, чтобы они имели нужное количество измерений. Первое измерение соответствует размеру батча, остальные — единичным размерам для совместимости с другими измерениями исходного тензора.


#### 8 из 11 шагов 

Нормализация "по инстансу" была изначально разработана для задачи style transfer. Идея, лежащая в основе этого слоя, что сеть должна быть независимой от контраста отдельных каналов исходного изображения.

На этом шаге вам предлагается реализовать нормализацию "по инстансу" без использования стандартного слоя со следующими упрощениями:
- Параметр Бета = 0.
- Параметр Гамма = 1.
- На вход подается трехмерный тензор (размер батча, число каналов, длина каждого канала инстанса).
- Требуется реализация только этапа обучения.

В слое нормализации "по инстансу" статистики считаются по последней размерности (по каждому входному каналу каждого входного примера).

Sample Input: anything

Sample Output: True

In [22]:
import torch
import torch.nn as nn

eps = 1e-3

batch_size = 5
input_channels = 2
input_length = 30

instance_norm = nn.InstanceNorm1d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, input_length, dtype=torch.float)


def custom_instance_norm1d(input_tensor, eps):
    # Количество каналов
    channels = input_tensor.size(1)
    
    # Список для хранения средних значений и дисперсий
    means = []
    vars = []
    
    # Проходим по каждому каналу
    for c in range(channels):
        # Берём срез по текущему каналу
        channel_slice = input_tensor[:, c]
        
        # Рассчитываем среднее значение по длине канала
        mean = torch.mean(channel_slice, dim=-1, keepdim=True)
        means.append(mean)
        
        # Рассчитываем дисперсию по длине канала
        var = torch.var(channel_slice, dim=-1, unbiased=False, keepdim=True)
        vars.append(var)
    
    # Преобразуем списки в тензоры
    means_tensor = torch.cat(means, dim=1)
    vars_tensor = torch.cat(vars, dim=1)
    
    # Расширяем размеры тензоров до нужного вида
    means_tensor = means_tensor.unsqueeze(-1)
    vars_tensor = vars_tensor.unsqueeze(-1)
    
    # Выполним нормализацию
    normed_tensor = (input_tensor - means_tensor) / torch.sqrt(vars_tensor + eps)
    
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = instance_norm(input_tensor)
custom_output = custom_instance_norm1d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output, atol=1e-06) and norm_output.shape == custom_output.shape)

True


Этот код реализует нормализацию "по инстансу" для трёхмерного тензора, где статистика считается по последнему измерению (длина каждого канала инстанса).